<a href="https://colab.research.google.com/github/THESHAMIK/Real-Life-Projects/blob/main/Deep_NN/KHDS_Keras_%2B_sklearn_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Try to Optimize Neural nw for KHDS. To tune hyper params straight try to use Bayesian Optmizer than Grid Search & Random Search as Bayesian is much better than both. 

Will use Bayesian to TUNE : 1. Learning Rate, 2. Batch Size 3. Epoch 4. No of Neurons to be used 5. Layer in model Arch can also be found!

neurons, activation, optimizer, learning_rate,  batch_size, epochs 

https://www.analyticsvidhya.com/blog/2021/05/tuning-the-hyperparameters-and-layers-of-neural-network-deep-learning/

In [ ]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=58fb1d4ec2165fdde8d93058059d24f8b45b98dde858e6a5a2d3964ecc928b8b
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


In [ ]:
# Import packages
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
#from keras.optimizers import SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.optimizers import SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.optimizers import Adam ##because cannot import name 'Adam' from 'keras.optimizers'
from keras.callbacks import EarlyStopping, ModelCheckpoint
#from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)

import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

In [ ]:
df = pd.read_csv("khds_final_1.csv")

In [ ]:
#U can see last column Product Sulpur(Y) is an object & not Float
#data['Product Sulphur']=data['Product Sulphur'].astype(float64) ##==> # This is NOT working
df['Product Sulphur'] = pd.to_numeric(df['Product Sulphur'],errors = 'coerce') ##==> This one works as pd.to_numeric works for convert a list, a series, an array, or a tuple to a numeric datatype
#type(data['Product Sulphur']) ##This shows that Product Sulphur data type is a series hence astype did not work
df.dtypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711 entries, 0 to 710
Data columns (total 34 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   LABfeed Sulphur                            711 non-null    float64
 1   T1 : FRESH FEED FLOW TO KHDS KGPerHour     711 non-null    float64
 2   T1 : Reactor inlet temperature DEGC        711 non-null    float64
 3   T1 : Hydrogen make-up flow NM3perHour      711 non-null    float64
 4   T1 : Recycle hydrogen flow NM3perHour      711 non-null    float64
 5   T1 : REACTOR OUTLET Temp DEGC              711 non-null    float64
 6   T1 : REACTOR FEED INLET Pressure KGperCM2  711 non-null    float64
 7   T1 : Cold separator pressure KGperCM2      711 non-null    float64
 8   T1 : KERO FEED DENSITY KGperCM2            711 non-null    float64
 9   T2 : FRESH FEED FLOW TO KHDS KGPerHour     711 non-null    float64
 10  T2 : Reactor inlet tempera

In [ ]:
filtered_df = df[df['Product Sulphur'].notnull()]

In [ ]:
df.dropna(axis=0)

,LABfeed Sulphur,T1 : FRESH FEED FLOW TO KHDS KGPerHour,T1 : Reactor inlet temperature DEGC,T1 : Hydrogen make-up flow NM3perHour,T1 : Recycle hydrogen flow NM3perHour,T1 : REACTOR OUTLET Temp DEGC,T1 : REACTOR FEED INLET Pressure KGperCM2,T1 : Cold separator pressure KGperCM2,T1 : KERO FEED DENSITY KGperCM2,T2 : FRESH FEED FLOW TO KHDS KGPerHour,T2 : Reactor inlet temperature DEGC,T2 : Hydrogen make-up flow NM3perHour,T2 : Recycle hydrogen flow NM3perHour,T2 : REACTOR OUTLET Temp DEGC,T2 : REACTOR FEED INLET Pressure KGperCM2,T2 : Cold separator pressure KGperCM2,T2 : KERO FEED DENSITY KGperCM2,T3 : FRESH FEED FLOW TO KHDS KGPerHour,T3 : Reactor inlet temperature DEGC,T3 : Hydrogen make-up flow NM3perHour,T3 : Recycle hydrogen flow NM3perHour,T3 : REACTOR OUTLET Temp DEGC,T3 : REACTOR FEED INLET Pressure KGperCM2,T3 : Cold separator pressure KGperCM2,T3 : KERO FEED DENSITY KGperCM2,T4 : FRESH FEED FLOW TO KHDS KGPerHour,T4 : Reactor inlet temperature DEGC,T4 : Hydrogen make-up flow NM3perHour,T4 : Recycle hydrogen flow NM3perHour,T4 : REACTOR OUTLET Temp DEGC,T4 : REACTOR FEED INLET Pressure KGperCM2,T4 : Cold separator pressure KGperCM2,T4 : KERO FEED DENSITY KGperCM2,Product Sulphur
0,1000.0,72482.43948,288.485914,685.146132,11114.58673,288.637635,21.927602,18.775248,0.000000,72337.37151,287.651367,740.221182,11041.21585,288.761036,21.810156,18.688527,0.000000,72376.86708,287.823835,660.058960,11072.44088,287.738775,21.813427,18.693064,0.000000,72353.77432,287.978641,614.163076,11226.93106,288.712160,21.805388,18.666758,0.000000,2.0
1,1400.0,75588.69117,286.406314,571.248477,11259.14433,286.921575,21.911739,18.702087,0.000000,75746.39266,286.441067,549.330129,11228.98135,286.630241,21.870260,18.674408,0.000000,75963.37102,286.687275,617.336266,11004.25626,286.128780,21.711249,18.533493,0.000000,75918.19844,287.134693,464.921691,11066.97566,286.615329,21.934672,18.747208,0.000000,0.5
2,1300.0,75675.29792,284.035734,564.716667,10668.45313,284.140627,21.900282,18.734029,0.000000,75628.86875,284.037801,582.175509,10721.20293,284.517944,21.882274,18.724772,0.000000,75646.63073,283.697494,559.376522,10763.02799,284.325244,21.813410,18.677979,0.000000,76002.57969,284.800492,580.681543,10701.71289,284.840621,21.848295,18.704427,0.000000,3.0
3,1500.0,46269.02109,276.210773,371.162782,14497.57829,276.808471,21.747162,19.528809,0.000000,46001.59297,276.116185,527.354706,14646.65260,277.295914,21.531470,19.330088,0.000000,46411.94089,275.763115,258.167509,14658.63428,277.148141,21.664635,19.450734,0.000000,46176.64115,275.822623,565.165267,14783.88027,276.540142,21.547435,19.357554,0.000000,4.0
4,1400.0,45394.43555,276.126803,296.835800,14606.11231,276.776767,21.560785,19.492783,0.000000,45231.54465,276.010997,441.971037,14684.18629,276.807028,21.579045,19.508255,0.000000,45301.01027,276.366341,365.445542,14693.11828,276.877170,21.526057,19.478851,0.000000,45265.78914,276.090728,409.074912,14675.20297,277.132889,21.597432,19.496689,0.000000,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,1100.0,60139.03926,269.888446,511.399785,12541.86891,271.360266,21.023166,18.624649,694.466242,59989.55111,270.211539,540.839133,12484.32941,272.175701,21.005653,18.603541,694.511919,60149.79883,270.286752,554.881517,12506.45005,272.583015,21.021043,18.628108,695.525236,60417.56009,270.061212,581.807254,12470.92394,272.810638,20.995547,18.596115,694.614817,19.0
707,1200.0,60362.18822,269.971474,558.111740,12542.77303,272.554566,21.018436,18.601252,697.014856,60543.49408,270.135722,593.864419,12558.97292,272.846240,20.994644,18.566004,696.803418,60547.30573,269.939566,458.611999,12542.78582,272.927622,21.076758,18.634313,696.144514,60372.60352,270.042269,501.625794,12590.90794,272.603820,21.103575,18.656947,696.279553,20.0
708,1200.0,60425.38188,270.062989,528.413451,12517.83553,272.306049,21.014648,18.605937,693.293790,60522.48813,269.937830,543.695607,1

In [ ]:
#df.info()
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 709 entries, 0 to 710
Data columns (total 34 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   LABfeed Sulphur                            709 non-null    float64
 1   T1 : FRESH FEED FLOW TO KHDS KGPerHour     709 non-null    float64
 2   T1 : Reactor inlet temperature DEGC        709 non-null    float64
 3   T1 : Hydrogen make-up flow NM3perHour      709 non-null    float64
 4   T1 : Recycle hydrogen flow NM3perHour      709 non-null    float64
 5   T1 : REACTOR OUTLET Temp DEGC              709 non-null    float64
 6   T1 : REACTOR FEED INLET Pressure KGperCM2  709 non-null    float64
 7   T1 : Cold separator pressure KGperCM2      709 non-null    float64
 8   T1 : KERO FEED DENSITY KGperCM2            709 non-null    float64
 9   T2 : FRESH FEED FLOW TO KHDS KGPerHour     709 non-null    float64
 10  T2 : Reactor inlet tempera

In [ ]:
target_col = "Product Sulphur"
X = filtered_df.loc[:, filtered_df.columns != target_col]
y = filtered_df.loc[:, target_col]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    random_state=2021)

In [ ]:
# Create function
def nn_cl_bo(neurons, activation, optimizer, learning_rate,  batch_size, epochs ):
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                 'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                 'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                 'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    neurons = round(neurons)
    activation = activationL[round(activation)]
    batch_size = round(batch_size)
    epochs = round(epochs)
    def nn_cl_fun():
        opt = Adam(lr = learning_rate)
        nn = Sequential()
        nn.add(Dense(neurons, input_dim=33, activation=activation))
        nn.add(Dense(neurons, activation=activation))
        nn.add(Dense(1))
        nn.compile(loss='mean_absolute_error', optimizer=opt, metrics=['mean_squared_error'])
        return nn
    es = EarlyStopping(monitor='mean_squared_error', mode='max', verbose=0, patience=20)
    nn = KerasRegressor(build_fn=nn_cl_fun, epochs=epochs, batch_size=batch_size,
                         verbose=0)
    #kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    kfold = 5
    #score = cross_val_score(nn, X_train, y_train, scoring=r2, cv=kfold, fit_params={'callbacks':[es]}).mean()
    score = cross_val_score(nn, X_train, y_train, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score

In [ ]:
# Set paramaters
params_nn ={
    'neurons': (10, 100),
    'activation':(0, 9),
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'batch_size':(10, 100),
    'epochs':(20, 100)
}
# Run Bayesian Optimization
nn_bo = BayesianOptimization(nn_cl_bo, params_nn, random_state=111)
nn_bo.maximize(init_points=20, n_iter=4)

|   iter    |  target   | activa... | batch_... |  epochs   | learni... |  neurons  | optimizer |
-------------------------------------------------------------------------------------------------
|  1        | -3.225    |  5.51     |  25.22    |  54.88    |  0.7716   |  36.58    |  1.044    |
|  2        | -2.02     |  0.2023   |  47.82    |  39.09    |  0.3443   |  99.16    |  1.664    |
|  3        | -2.261    |  0.7307   |  70.26    |  69.7     |  0.2815   |  51.96    |  0.8286   |
|  4        | -2.725    |  0.6656   |  91.07    |  83.52    |  0.8422   |  83.37    |  6.937    |
|  5        | -2.291    |  5.195    |  83.24    |  53.71    |  0.03717  |  50.87    |  0.7373   |
|  6        |  nan      |  7.355    |  72.8     |  65.22    |  0.2815   |  99.86    |  0.9663   |
|  7        | -81.63    |  5.539    |  53.65    |  52.4     |  0.7306   |  39.05    |  2.804    |
|  8        | -2.484    |  2.871    |  95.25    |  93.5     |  0.8157   |  13.07    |  6.604    |
|  9        | -5.513

ValueError: ignored

**Here are the best hyperparameters.**

In [ ]:
params_nn_ = nn_bo.max['params']
activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
               'elu', 'exponential', LeakyReLU,'relu']
params_nn_['activation'] = activationL[round(params_nn_['activation'])]
params_nn_

{'activation': 'exponential',
 'batch_size': 72.79549611360693,
 'epochs': 65.22283210717812,
 'learning_rate': 0.28148502249402685,
 'neurons': 99.86263693206844,
 'optimizer': 0.9662943713915518}

In [ ]:
import tensorflow as tf
import keras
from keras import layers
from keras import models
from keras import utils
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Activation
from keras.regularizers import l2
#from keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
#from keras.optimizers import RMSprop
from keras import datasets

from keras.callbacks import LearningRateScheduler
from keras.callbacks import History

from keras import losses
from sklearn.utils import shuffle


In [ ]:
epochs=60
learning_rate = 0.1
decay_rate = learning_rate / epochs
momentum = 0.8

sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

In [ ]:
# build the model
input_dim = X_train.shape[1]

lr_model = Sequential()
lr_model.add(Dense(64, input_shape=(33,), activation=tf.nn.relu, kernel_initializer='uniform')) 
lr_model.add(Dropout(0.1))
lr_model.add(Dense(64, kernel_initializer='uniform', activation=tf.nn.relu))
lr_model.add(Dense(1, kernel_initializer='uniform'))

# compile the model
lr_model.compile(loss='mean_squared_error',optimizer=sgd)

In [ ]:
%%time
# Fit the model
batch_size = int(input_dim/100)

lr_model_history = lr_model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_test, y_test))

Epoch 1/60
18/18 [==============================] - 1s 12ms/step - loss: nan - val_loss: nan
Epoch 2/60
18/18 [==============================] - 0s 5ms/step - loss: nan - val_loss: nan
Epoch 3/60
18/18 [==============================] - 0s 4ms/step - loss: nan - val_loss: nan
Epoch 4/60
18/18 [==============================] - 0s 3ms/step - loss: nan - val_loss: nan
Epoch 5/60
18/18 [==============================] - 0s 3ms/step - loss: nan - val_loss: nan
Epoch 6/60
18/18 [==============================] - 0s 3ms/step - loss: nan - val_loss: nan
Epoch 7/60
18/18 [==============================] - 0s 4ms/step - loss: nan - val_loss: nan
Epoch 8/60
18/18 [==============================] - 0s 3ms/step - loss: nan - val_loss: nan
Epoch 9/60
18/18 [==============================] - 0s 4ms/step - loss: nan - val_loss: nan
Epoch 10/60
18/18 [==============================] - 0s 3ms/step - loss: nan - val_loss: nan
Epoch 11/60
18/18 [==============================] - 0s 5ms/step - loss: nan -